<a href="https://colab.research.google.com/github/Bayrem-ben/colab.github.io/blob/main/extract_categ_from_jort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import cv2
import imutils
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import re

In [2]:
from urllib.request import Request, urlopen
import json

import pandas as pd    

file = open("/content/drive/MyDrive/jort/jortapi.json", 'r')
data = file.read()
#data = json.load(file)
file.close()

In [ ]:
import ast

#res = json.loads(data)
res = ast.literal_eval(data)


print(len(res))

In [ ]:
df = pd.DataFrame(data, columns=['article_text, lang'])

In [ ]:
#a_json = json.loads("/content/drive/MyDrive/jort/jortapi.json")
#df = pd.read_json("/content/drive/MyDrive/jort/jortapi.json", orient='columns')
df= pd.read_csv("/content/drive/MyDrive/jort/jortapi.csv")
df

In [8]:
import csv

file = open('/content/drive/MyDrive/jort/jortapi.csv')

csvreader = csv.reader(file)
#header = next(csvreader)
#print(header)
rows = []
for row in csvreader:
    rows.append(row)
#print(rows)
file.close()

In [15]:
df = pd.DataFrame(rows)

In [ ]:
df.head(100)

In [110]:
update_df = df.drop([df.index[0], df.index[1] , df.index[2]]) #
update_df = update_df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 8, 10,11,12,13,14,15,16,17,18]], axis=1)

In [111]:
update_df = update_df.dropna()
update_df = update_df[(update_df[9] == "Fr")]
update_df

,7,9
3,N° 1 ACTES JUDICIAIRES VENTES AUX ENCHERES PU...,Fr
4,Vente aux enchères publiques sur saisie immob...,Fr
5,Vente aux enchères publiques sur saisie immob...,Fr
6,Vente aux enchères publiques d'un immeuble im...,Fr
7,Vente aux enchères publiques Maître Mohamed F...,Fr
...,...,...
503047,Avis Monsieur Hatem Baccar déclare par la pré...,Fr
503048,Avis Monsieur Walid Baklouti déclare par la p...,Fr
503049,Avis Les héritiers de Ahmed Hdiji annonce qu'...,Fr
503050,Rectificatifs Rectificatif Rectificatif de l'...,Fr


#start

In [ ]:
def clean_txt(text):
  txt = re.sub("\W+", ' ', str(text)) #\W+ 
  txt = re.sub("[?|$|.|!]", '', str(txt))
  txt = re.sub("صفحـة", '', str(txt))
  txt = re.sub("عدد", '', str(txt))
  txt = re.sub("إعلانات قانونية وشرعية وعدلية", '', str(txt))
  txt = re.sub("الرائد الرسمي للجمهورية التونسية", '', str(txt))
  txt = re.sub("–", '', str(txt))
  txt = re.sub("تعريف الإمضاء : رئيس البلدية", '', str(txt))
  txt = re.sub("تعريف الإمضاء رئيس البلدية", '', str(txt))
  txt = re.sub("نسخة مطابقة : الرئيس المدير العام للمطبعة الرسمية للجمهورية التونسية", '', str(txt))
  txt = re.sub("نسخة مطابقة الرئيس المدير العام للمطبعة الرسمية للجمهورية التونسية", '', str(txt))
  txt = re.sub("ر د م د", '', str(txt))

  return txt

In [ ]:
txts = '/content/drive/MyDrive/press/99.txt'
with open(txts, 'r+') as f:
    lines = f.read() #readlines  .rstrip('\n')

In [ ]:


##############

regex = "/202[A-Z,0-9]{10}|202[A-Z,0-9]{11}|202[A-Z,0-9]{12}|202[A-Z,0-9]{13}|202[A-Z,0-9]{14}/"

xtxt = re.split(regex, str(lines))

i = 0
w1 = 0
w2 = 0
w3 = 0
w4 = 0

lis_commercial = []
lis_auction = []
lis_societe = []
lis_annonce = []

words1 = ['أصول تجارية','أصل تجاري']

words2 = ['بتة','المزاد العلني','بالمزاد العلني','علان تبتيت','عقلة عقارية','رهن الإختياري','المعقول عنه','المعقول ضده']

words3 = ['التصرف في شركات','إعلان للدائنين','رأس المال','شركة خفية الإسم','مصادقة على تقارير','أشركات خرى','رأس مال شركة'
          ,'رأس مالها','حل الشركة','استدعاء للجلسة العامة العادية','ستدعاء للجلسة العامة الخارقة'
          ,'فتح إجراءات التسوية القضائية','دائن','إعلان توزيع ','ختم أعمال الفلسة','علان توزيع أموال','إعلان للمساهمين','تعيين مجلس إدارة','طلب عروض','فلسة']

words4 = ['جمعيات','جمعية','أحزاب','نقابات','نقابة','اتحاد','حزب','تكوين ودادية','انتخاب أعضاء'
          ,'تكوين جمعية','انتخاب مكتب جديد','ضياع سند','إعــلانات للعمــوم','أضاعا','أضاع'
          ,'اعلام للعموم','ضياع','إصلاح خطأ','تكوين نقابة ','تغيير إسم جمعية ','إستخراج رسم','حكم تقديم','يعلن ورثة المرحوم','يعلن ورثة المرحومة','رسم تملك مورثهم']

mat_re1 = []
mat_re2 = []

for x in xtxt :
  x = clean_txt(x)
  #print('section : ', i)
  #print(x)
  i+=1

  for w in words1:
    if (w in x) :
      lis_commercial.append(x)
      w1+=1
      break
      
  for w in words2:
    if (w in x) :
      lis_auction.append(x)
      w2+=1
      break
  
  for w in words3:
    words = words1 + words2 + words4
    #matching = [s for s in words if s in lis_societe]
    for wo in words:
      if w in x and wo not in x:
        lis_societe.append(x)
        w3+=1

  for w in words4:
    words = words1 + words2 + words3
    for wo in words:
      if w in x and wo not in x :
        lis_annonce.append(x)
        w4+=1

lis_societe = list(dict.fromkeys(lis_societe))
lis_annonce = list(dict.fromkeys(lis_annonce))

word_3 = words1 + words2 + words4
matching3 = [s for s in word_3 if s in lis_societe]
print(matching3)
word_4 = words1 + words2 + words3
matching4 = [s for s in word_4 if s in lis_annonce]
print(matching4)

print('i : ',i)
print('commercial : ',w1,len(lis_commercial)) 
print('auction : ',w2,len(lis_auction))
print('societe : ',w3,len(lis_societe))
print('annonce : ',w4,len(lis_annonce)) 

[]
[]
i :  92
commercial :  31 31
auction :  9 9
societe :  2532 42
annonce :  658 12


In [ ]:
matching = [s for s in lis_commercial if s in lis_societe]
print(matching)

['3 _________________________ استدعاء للجلسة العامة العادية الشركة التونسية لصناعة الموس والحشايا ومشتقاته الحديثة ش خ إ رأس مالها 000 148 16 دينار طريق قابس كلم 3 صفاقس م و 0620656M إن السادة المساهمين في رأس مال الشركة التونسية لصناعة الموس والحشايا ومشتقاته الحديثة مدعوون لحضور الجلسة العامة العادية المزمع عقدها يوم الجمعة 26 نوفمبر 2021 على الساعة التاسعة صباحا بالمقر الاجتماعي للشركة قصد التداول في جدول الأعمال التالي ـ تلاوة تقرير التصرف لسنة 2020 ـ تلاوة تقريري مراقب الحسابات لسنة 2020  2420 الرائد الرسمي للجمهورية التونسية  29 أكتوبر 2021 عــدد 99 ـ المصادقة على حسابات السنة المالية 2020 ـ تخصيص النتيجة النهائية للسنة المالية 2020 ـ إعطاء الإبراء التام للوكالة إزاء أعمال تصرف سنة 2020 ـ المصادقة على الاتفاقيات المبرمة طبقا لمقتضيات الفصل 115 من مجلة الشركات التجارية ـ إصلاح خطأ مادي ـ مسائل مختلفة ']


#fun

In [ ]:
def clean_txt(text):
  txt = re.sub("\W+", ' ', str(text)) #\W+ 
  txt = re.sub("[?|$|.|!]", '', str(txt))
  txt = re.sub("صفحـة", '', str(txt))
  txt = re.sub("عدد", '', str(txt))
  txt = re.sub("إعلانات قانونية وشرعية وعدلية", '', str(txt))
  txt = re.sub("الرائد الرسمي للجمهورية التونسية", '', str(txt))
  txt = re.sub("–", '', str(txt))
  txt = re.sub("تعريف الإمضاء : رئيس البلدية", '', str(txt))
  txt = re.sub("تعريف الإمضاء رئيس البلدية", '', str(txt))
  txt = re.sub("نسخة مطابقة : الرئيس المدير العام للمطبعة الرسمية للجمهورية التونسية", '', str(txt))
  txt = re.sub("نسخة مطابقة الرئيس المدير العام للمطبعة الرسمية للجمهورية التونسية", '', str(txt))
  txt = re.sub("ر د م د", '', str(txt))
  txt = re.sub("[0-9]", '', str(txt))

  return txt

def count(path):
  i = 0
  w1 = 0
  w2 = 0
  w3 = 0
  w4 = 0

  lis_commercial = []
  lis_auction = []
  lis_societe = []
  lis_annonce = []

  words1 = ['أصول تجارية','أصل تجاري']
  words2 = ['بتة','المزاد العلني','بالمزاد العلني','علان تبتيت','عقلة عقارية','رهن الإختياري','المعقول عنه','المعقول ضده']
  words3 = ['التصرف في شركات','إعلان للدائنين','رأس المال','شركة خفية الإسم','مصادقة على تقارير','أشركات خرى','رأس مال شركة'
                  ,'رأس مالها','حل الشركة','استدعاء للجلسة العامة العادية','ستدعاء للجلسة العامة الخارقة'
                  ,'فتح إجراءات التسوية القضائية','دائن','إعلان توزيع ','ختم أعمال الفلسة','علان توزيع أموال','إعلان للمساهمين','تعيين مجلس إدارة','طلب عروض','فلسة']
  words4 = ['جمعيات','جمعية','أحزاب','نقابات','نقابة','اتحاد','حزب','تكوين ودادية','انتخاب أعضاء'
                  ,'تكوين جمعية','انتخاب مكتب جديد','ضياع سند','إعــلانات للعمــوم','أضاعا','أضاع'
                  ,'اعلام للعموم','ضياع','إصلاح خطأ','تكوين نقابة ','تغيير إسم جمعية ','إستخراج رسم','حكم تقديم','يعلن ورثة المرحوم','يعلن ورثة المرحومة','رسم تملك مورثهم']

  mat_w3 = []
  mat_re2 = []

  for fname in os.listdir(path):
        fpath = os.path.join(path, fname)

        with open(fpath, 'r+') as f:
            lines = f.read() #readlines  .rstrip('\n')   
        regex = "/201[A-Z,0-9]{9}|201[A-Z,0-9]{10}|201[A-Z,0-9]{11}|201[A-Z,0-9]{12}|201[A-Z,0-9]{13}|201[A-Z,0-9]{14}|201[A-Z,0-9]{15}|202[A-Z,0-9]{09}|202[A-Z,0-9]{10}|202[A-Z,0-9]{11}|202[A-Z,0-9]{12}|202[A-Z,0-9]{13}|202[A-Z,0-9]{14}|202[A-Z,0-9]{15}/"
        xtxt = re.split(regex, str(lines))

        for x in xtxt :
          x = clean_txt(x)
          i+=1
          for w in words1:
            words = words3 + words2 + words4
            #matching = [s for s in words if s in lis_societe]
            for wo in words:
              if w in x and wo not in x:
                lis_commercial.append(x)
                w1+=1
          for w in words2:
            words = words1 + words3 + words4
            #matching = [s for s in words if s in lis_societe]
            for wo in words:
              if w in x and wo not in x:
                lis_auction.append(x)
                w2+=1

          for w in words3:
            words = words1 + words2 + words4
            for wo in words:
              if w in x and wo not in x :
                lis_societe.append(x)
                w3+=1

          for w in words4:
            words = words1 + words2 + words3
            for wo in words:
              if w in x and wo not in x :
                lis_annonce.append(x)
                w4+=1

  lis_commercial = list(dict.fromkeys(lis_commercial))
  lis_auction = list(dict.fromkeys(lis_auction))
  lis_societe = list(dict.fromkeys(lis_societe))
  lis_annonce = list(dict.fromkeys(lis_annonce))

  print('i : ',i)
  print('commercial : ',w1,len(lis_commercial)) 
  print('auction : ',w2,len(lis_auction))
  print('societe : ',w3,len(lis_societe))
  print('annonce : ',w4,len(lis_annonce))  
  
  return lis_commercial, lis_auction, lis_societe, lis_annonce

lien = '/content/drive/MyDrive/press/jort_txt/'
l1,l2,l3,l4 = count(lien)

i :  10748
commercial :  171727 2879
auction :  184983 1065
societe :  294737 4241
annonce :  89796 1670


In [ ]:
l1 = list(set(l1) - set(l2+l4))
len(l1)

2843

In [ ]:
l3 = list(set(l3) - set(l1+l2+l4))
len(l3)


3079

In [ ]:
l4 = list(set(l4) - set(l3+l2+l1))
len(l4)

1627

create 4 dataframe + 4 csv files

In [ ]:
# Initialize data to lists.
data = l1

# Creates DataFrame.
df1 = pd.DataFrame(data, columns=['أصل تجاري'])
  
# Print the data
df1

In [ ]:
data = l2

# Creates DataFrame.
df2 = pd.DataFrame(data, columns=['بيع بالمزاد العلني'])
  
# Print the data
df2

In [ ]:
data = l3

# Creates DataFrame.
df3 = pd.DataFrame(data, columns=['الشركة طلب العروض'])
  
# Print the data
df3

In [ ]:
data = l4

# Creates DataFrame.
df4 = pd.DataFrame(data, columns=['إعلان'])
  
# Print the data
df4

In [ ]:
from sklearn.utils import shuffle

df1 = shuffle(df1)
df2 = shuffle(df2)
df3 = shuffle(df3)
df4 = shuffle(df4)

In [ ]:
df1.to_csv(r'/content/drive/MyDrive/press/dataset_asl.csv', index = False)

In [ ]:
df2.to_csv(r'/content/drive/MyDrive/press/dataset_mazed.csv', index = False)

In [ ]:
df3.to_csv(r'/content/drive/MyDrive/press/dataset_appel.csv', index = False)

In [ ]:
df4.to_csv(r'/content/drive/MyDrive/press/dataset_announce.csv', index = False)

In [ ]:
# Initialize data to lists.
data = [l1,l2,l3,l4]

# Creates DataFrame.
df = pd.DataFrame(data, index=['أصل تجاري','بيع بالمزاد العلني','الشركة طلب العروض','إعلان'])
  
# Print the data
df

In [ ]:
df.to_csv(r'/content/drive/MyDrive/press/dataset.csv', index = False)

In [ ]:
"""
أشركات خرى

التصرف في شركات
مصادقة على تقارير ........................................................................
أصول تجارية
بيع أصول تجارية ..........................................................................
كراء أصول تجارية .........................................................................
جمعيات وأحزاب ونقابات
تكوين ....................................................................................
مصادقة على تقارير ........................................................................
إجراءات عدلية
بيوعات بالمزاد العلني ......................................................................
إعلان للدائنين .............................................................................
استدعاءات
شركات خفية الإسم وشركات ذات مسؤولية محدودة وشركات أخرى ............................
مختلفات
إعلانات للعموم ............................................................................
موازنات وحسابات مالية
موازنة سيكاف الإنماء.......................................................................
موازنة سيكاف تثمير........................................................................
إجراءات إدارية
إعلانات للعموم ......................................................................................
إعلانات للعموم وإصلاح خطأ.................................................................. 
"""